In [1]:
import pandas as pd
from scipy import stats

In [2]:
structure_info = pd.read_excel('structure_info.xlsx', index_col=0)

In [3]:
fa_f_data = structure_info[(structure_info['MouseGender'] == 'f') & (structure_info['EnvironType'] == 'fa')]
fa_m_data = structure_info[(structure_info['MouseGender'] == 'm') & (structure_info['EnvironType'] == 'fa')]
cap_f_data = structure_info[(structure_info['MouseGender'] == 'f') & (structure_info['EnvironType'] == 'cap')]
cap_m_data = structure_info[(structure_info['MouseGender'] == 'm') & (structure_info['EnvironType'] == 'cap')]

In [4]:
def static_and_test(key):
    _, pval = stats.normaltest(structure_info[key])
    if pval > 0.05:
        test = stats.ttest_ind
    else:
        test = stats.ranksums

    [fa_f_cap_f, fa_m_cap_m, cap_f_cap_m, fa_f_fa_m] = list(map(
        lambda x, y: test(x, y)[1], 
        [fa_f_data[key], fa_m_data[key], cap_f_data[key], fa_f_data[key]], 
        [cap_f_data[key], cap_m_data[key], cap_m_data[key], fa_m_data[key]]
    ))

    [fa_f_mean, fa_m_mean, cap_f_mean, cap_m_mean] = list(map(
        lambda x: x.mean() / (20 ** 3), 
        [fa_f_data[key], fa_m_data[key], cap_f_data[key], cap_m_data[key]]
    ))

    [fa_f_std, fa_m_std, cap_f_std, cap_m_std] = list(map(
        lambda x: x.std() / (20 ** 3), 
        [fa_f_data[key], fa_m_data[key], cap_f_data[key], cap_m_data[key]]
    ))

    static_dict = {
        'region': key,

        'fa_f_cap_f': fa_f_cap_f,
        'fa_m_cap_m': fa_m_cap_m,
        'cap_f_cap_m': cap_f_cap_m,
        'fa_f_fa_m': fa_f_fa_m,

        'fa_f_mean': fa_f_mean,
        'fa_m_mean': fa_m_mean,
        'cap_f_mean': cap_f_mean,
        'cap_m_mean': cap_m_mean,

        'fa_f_std': fa_f_std,
        'fa_m_std': fa_m_std,
        'cap_f_std': cap_f_std,
        'cap_m_std': cap_m_std,
    }
    return static_dict

In [5]:
region_keys = list(structure_info.keys()[8:])

In [6]:
static_info = []
for key in region_keys:
    static_info.append(static_and_test(key))

In [7]:
static_df = pd.DataFrame(static_info)

In [8]:
static_df.to_excel('vbm_static.xlsx')

In [9]:
# list(map(
#     lambda x: static_df[static_df[x] < 0.05].to_excel('vbm_{}.xlsx'.format(x)),
#     ['fa_f_cap_f', 'fa_m_cap_m', 'cap_f_cap_m', 'fa_f_fa_m']
# ))
# static_df[
#     (static_df['fa_f_cap_f'] < 0.05) | (static_df['fa_m_cap_m'] < 0.05) | (static_df['cap_f_cap_m'] < 0.05) | (static_df['fa_f_fa_m'] < 0.05)
# ].to_excel('vbm_static_significance.xlsx')